In [11]:
import pandas as pd

coverage = pd.read_csv("../input/spring-petclinic/jacoco.csv")
coverage = coverage_data[['PACKAGE', 'CLASS', 'LINE_MISSED', 'LINE_COVERED']]
coverage.head()

,PACKAGE,CLASS,LINE_MISSED,LINE_COVERED
0,org.springframework.samples.petclinic,PetclinicInitializer,0,24
1,org.springframework.samples.petclinic.model,NamedEntity,1,4
2,org.springframework.samples.petclinic.model,Specialty,0,1
3,org.springframework.samples.petclinic.model,PetType,0,1
4,org.springframework.samples.petclinic.model,Vets,4,0


In [12]:
coverage['LINES'] = coverage.LINE_COVERED + coverage.LINE_MISSED
coverage['RATIO'] = coverage.LINE_COVERED / coverage.LINES
coverage['fqn'] = coverage.PACKAGE + "." + coverage.CLASS
coverage.head()

,PACKAGE,CLASS,LINE_MISSED,LINE_COVERED,LINES,RATIO,fqn
0,org.springframework.samples.petclinic,PetclinicInitializer,0,24,24,1.0,org.springframework.samples.petclinic.Petclini...
1,org.springframework.samples.petclinic.model,NamedEntity,1,4,5,0.8,org.springframework.samples.petclinic.model.Na...
2,org.springframework.samples.petclinic.model,Specialty,0,1,1,1.0,org.springframework.samples.petclinic.model.Sp...
3,org.springframework.samples.petclinic.model,PetType,0,1,1,1.0,org.springframework.samples.petclinic.model.Pe...
4,org.springframework.samples.petclinic.model,Vets,4,0,4,0.0,org.springframework.samples.petclinic.model.Vets


In [13]:
import py2neo

graph = py2neo.Graph()
graph

<Graph uri='http://localhost:7474/db/data/'>

In [14]:
query="""
OPTIONAL MATCH (c:Class)-[:BELONGS_TO]->(s:Subdomain)
WHERE HAS(c.fqn)
RETURN c.fqn as fqn, s.name as subdomain
"""

classes = pd.DataFrame(graph.data(query))
classes.head()

,fqn,subdomain
0,org.springframework.samples.petclinic.service....,Clinic
1,org.springframework.samples.petclinic.service....,Clinic
2,org.springframework.samples.petclinic.service....,Clinic
3,org.springframework.samples.petclinic.service....,Clinic
4,org.springframework.samples.petclinic.service....,Clinic


In [18]:
class_coverage = pd.merge(classes, coverage, left_on='fqn', right_on='fqn', how='left')
class_coverage['RATIO'] = class_coverage['RATIO'].fillna(0)
class_coverage.head()

,fqn,subdomain,PACKAGE,CLASS,LINE_MISSED,LINE_COVERED,LINES,RATIO
0,org.springframework.samples.petclinic.service....,Clinic,org.springframework.samples.petclinic.service,ClinicServiceImpl,2.0,16.0,18.0,0.888889
1,org.springframework.samples.petclinic.service....,Clinic,NaN,NaN,NaN,NaN,NaN,0.000000
2,org.springframework.samples.petclinic.service....,Clinic,NaN,NaN,NaN,NaN,NaN,0.000000
3,org.springframework.samples.petclinic.service....,Clinic,NaN,NaN,NaN,NaN,NaN,0.000000
4,org.springframework.samples.petclinic.service....,Clinic,NaN,NaN,NaN,NaN,NaN,0.000000


In [27]:
class_coverage[(class_coverage['RATIO']==0) & ~(class_coverage['fqn'].str.contains("Test"))][['fqn', 'RATIO']]

,fqn,RATIO
7,org.springframework.samples.petclinic.reposito...,0.0
15,org.springframework.samples.petclinic.reposito...,0.0
16,org.springframework.samples.petclinic.reposito...,0.0
17,org.springframework.samples.petclinic.reposito...,0.0
20,org.springframework.samples.petclinic.reposito...,0.0
27,org.springframework.samples.petclinic.reposito...,0.0
30,org.springframework.samples.petclinic.reposito...,0.0
32,org.springframework.samples.petclinic.model.Vets,0.0
33,org.springframework.samples.petclinic.reposito...,0.0
34,org.springframework.samples.petclinic.reposito...,0.0
